In [237]:
import pandas as pd

In [238]:
def format_csv2df(csv_file):
    # CSV読み込み
    cols = ['受渡日', 'ファンド名', '数量［口］', '単価', '受渡金額/(ポイント利用)[円]']
    df = pd.read_csv(csv_file, encoding='shift-jis', parse_dates=['受渡日'], usecols=cols)

    # str型をint型に変換 &ポイント利用(100)などの表記削除
    df['数量［口］'] = df['数量［口］'].str.replace(',', '').astype(int)
    df['単価'] = df['単価'].str.replace(',', '').astype(int)
    df['受渡金額/(ポイント利用)[円]'] = df['受渡金額/(ポイント利用)[円]'].str.replace('\(\d+\)', '', regex=True)
    df['受渡金額/(ポイント利用)[円]'] = df['受渡金額/(ポイント利用)[円]'].str.replace(',', '').astype(int)

    # ファンド名の表記ゆれ修正
    d = {'楽天・全世界株式インデックス・ファンド（楽天・バンガード・ファンド（全世界株式））': '楽天・全世界株式',
         '楽天・全世界株式インデックス・ファンド(楽天・VT)': '楽天・全世界株式',
         '楽天・全米株式インデックス・ファンド（楽天・バンガード・ファンド（全米株式））': '楽天・全米株式',
         '楽天・全米株式インデックス・ファンド(楽天・VTI)': '楽天・全米株式',
         '三井住友・DC年金バランス30(債券重点型)(マイパッケージ)': '三井住友・DC年金バランス30',
         'eMAXIS Slim 米国株式(S&P500)': 'eMAXIS Slim 米国株式(S&P500)',
         'eMAXIS Slim 新興国株式インデックス': 'eMAXIS Slim 新興国株式',
         'eMAXIS Slim 全世界株式(オール・カントリー)': 'eMAXIS Slim 全世界株式'
         }
    df['ファンド名'] = df['ファンド名'].replace(d)

    # 後の集計用に年月でまとめる
    df['受渡年月'] = df['受渡日'].dt.to_period('M')
    df = df[['受渡日', '受渡年月', 'ファンド名', '数量［口］', '単価', '受渡金額/(ポイント利用)[円]']]

    # カラム名簡易化
    df.columns = ['受渡日', '受渡年月', 'ファンド名', '数量', '単価', '受渡金額']

    return df

In [239]:
input = 'src/tradehistory(INVST)_20230619.csv'
df = format_csv2df(input)
df.head()

,受渡日,受渡年月,ファンド名,数量,単価,受渡金額
0,2021-12-14,2021-12,楽天・全世界株式,3074,16267,5000
1,2021-12-14,2021-12,楽天・全米株式,2548,19627,5000
2,2021-12-15,2021-12,eMAXIS Slim 新興国株式,2326,12897,3000
3,2021-12-14,2021-12,eMAXIS Slim 米国株式(S&P500),2684,18629,5000
4,2022-01-17,2022-01,楽天・全世界株式,2425,16497,4000


In [240]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   受渡日     111 non-null    datetime64[ns]
 1   受渡年月    111 non-null    period[M]     
 2   ファンド名   111 non-null    object        
 3   数量      111 non-null    int64         
 4   単価      111 non-null    int64         
 5   受渡金額    111 non-null    int64         
dtypes: datetime64[ns](1), int64(3), object(1), period[M](1)
memory usage: 5.3+ KB


In [241]:
# import pygwalker as pyg
# pyg.walk(df)

In [242]:
df.head()

,受渡日,受渡年月,ファンド名,数量,単価,受渡金額
0,2021-12-14,2021-12,楽天・全世界株式,3074,16267,5000
1,2021-12-14,2021-12,楽天・全米株式,2548,19627,5000
2,2021-12-15,2021-12,eMAXIS Slim 新興国株式,2326,12897,3000
3,2021-12-14,2021-12,eMAXIS Slim 米国株式(S&P500),2684,18629,5000
4,2022-01-17,2022-01,楽天・全世界株式,2425,16497,4000


In [243]:
# df.columns

# 2023/06/22 期間中の損益額、損益率を求める！

In [244]:
# 期間中のファンド毎の所持数量（口数）と総受渡金額(ファンド別総積立額)
df_by_found = df[['ファンド名', '数量', '受渡金額']].groupby('ファンド名').sum()
df_by_found

,数量,受渡金額
ファンド名,,
eMAXIS Slim 全世界株式,49207,83000
eMAXIS Slim 新興国株式,68500,86000
eMAXIS Slim 米国株式(S&P500),40413,77000
三井住友・DC年金バランス30,103797,170000
楽天・全世界株式,53592,88000
楽天・全米株式,39132,77000


In [245]:
# 期間最終月におけるファンド毎の資産価値
# df[['受渡日', '受渡年月', 'ファンド名', '数量', '単価', '受渡金額']].tail(7)

In [246]:
# 期間最終月を取得する
latest_month = df['受渡年月'].unique()[-1].strftime('%Y-%m')
# latest_month

df_latest_month = df[df['受渡年月'] == latest_month][['ファンド名', '単価']]
df_latest_month

,ファンド名,単価
105,楽天・全世界株式,18059
106,楽天・全米株式,21728
107,eMAXIS Slim 全世界株式,18616
108,eMAXIS Slim 新興国株式,13089
109,eMAXIS Slim 米国株式(S&P500),21232
110,三井住友・DC年金バランス30,17102


In [247]:
# df_by_foundにdf_latest_monthをマージ
df_latest = pd.merge(df_latest_month, df_by_found, on='ファンド名')
df_latest

,ファンド名,単価,数量,受渡金額
0,楽天・全世界株式,18059,53592,88000
1,楽天・全米株式,21728,39132,77000
2,eMAXIS Slim 全世界株式,18616,49207,83000
3,eMAXIS Slim 新興国株式,13089,68500,86000
4,eMAXIS Slim 米国株式(S&P500),21232,40413,77000
5,三井住友・DC年金バランス30,17102,103797,170000


In [248]:
# カラム名を分かりやすく変更
df_latest.columns = ['ファンド名', '最終月単価(円/1万口)', '所持数(口)', '受渡金額(円)']
df_latest

,ファンド名,最終月単価(円/1万口),所持数(口),受渡金額(円)
0,楽天・全世界株式,18059,53592,88000
1,楽天・全米株式,21728,39132,77000
2,eMAXIS Slim 全世界株式,18616,49207,83000
3,eMAXIS Slim 新興国株式,13089,68500,86000
4,eMAXIS Slim 米国株式(S&P500),21232,40413,77000
5,三井住友・DC年金バランス30,17102,103797,170000


In [249]:
# 最新単価 ✕ 所持数量から資産額
df_latest['資産価値(円)'] = (df_latest['最終月単価(円/1万口)'] / 10000) * df_latest['所持数(口)']

# 資産価値 - 受渡金額よりファンド毎の損益計算
df_latest['損益額(円)'] = df_latest['資産価値(円)'] - df_latest['受渡金額(円)']
df_latest['損益率(％)'] = round(
    ((df_latest['資産価値(円)'] - df_latest['受渡金額(円)']) / df_latest['受渡金額(円)']) * 100, 2)

df_latest

,ファンド名,最終月単価(円/1万口),所持数(口),受渡金額(円),資産価値(円),損益額(円),損益率(％)
0,楽天・全世界株式,18059,53592,88000,96781.7928,8781.7928,9.98
1,楽天・全米株式,21728,39132,77000,85026.0096,8026.0096,10.42
2,eMAXIS Slim 全世界株式,18616,49207,83000,91603.7512,8603.7512,10.37
3,eMAXIS Slim 新興国株式,13089,68500,86000,89659.6500,3659.6500,4.26
4,eMAXIS Slim 米国株式(S&P500),21232,40413,77000,85804.8816,8804.8816,11.43
5,三井住友・DC年金バランス30,17102,103797,170000,177513.6294,7513.6294,4.42


In [251]:
# 期間中の総資産価値
total_reserve = df_latest['受渡金額(円)'].sum()
total_asset_value = df_latest['資産価値(円)'].sum()
total_profit_amount = int(df_latest['損益額(円)'].sum())
total_profit_rate = round(((total_asset_value - total_reserve) / total_reserve) * 100, 2)

if total_profit_amount > 0:
    print(f'評価額：+{total_profit_amount}円, +{total_profit_rate}%')
else:
    print(f'評価額：-{total_profit_amount}円, -{total_profit_rate}%')

評価額：+45389円, +7.81%


## 評価額：+45389円, +7.81% （計算には最終購入時の単価）
## 楽天証券のマイページには +65196円, +11.22% （計算には最新版単価）